In [1]:
import torch, numpy as np
from PIL import Image
from tqdm import tqdm
from torchvision.transforms import ToPILImage
import sys
sys.path.append(".")

from modelvshuman.models.wrappers.pytorch import HyCoCLIPModel
from modelvshuman.utils import load_dataset
from hycoclip.lorentz import pairwise_dist
from modelvshuman.models.pytorch.clip.imagenet_classes import imagenet_classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


c:\Users\xjzb2\.conda\envs\hycoclip\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = HyCoCLIPModel("hycoclip", "cue-conflict")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.model.to(device)

c:\Users\xjzb2\.conda\envs\hycoclip\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


HyCoCLIP(
  (visual): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
  

In [3]:
ds = load_dataset("cue-conflict", split="val")
loader = ds.loader

In [4]:
params  = list(model.model.parameters())
tot     = sum(p.numel() for p in params)
nz      = sum((p.abs()>1e-6).sum().item() for p in params)
print("loaded params:", nz, "/", tot, f"({100*nz/tot:.1f}%)")

zs = model.zeroshot_weights
print("zeroshot_weights:", zs.shape,
      "min/max/mean:", zs.min(), zs.max(), zs.mean())
print("first norms:", zs.norm(dim=0)[:10])

loaded params: 84315215 / 85290372 (98.9%)
zeroshot_weights: torch.Size([512, 1000]) min/max/mean: tensor(-0.2459, device='cuda:0') tensor(0.2461, device='cuda:0') tensor(0.0002, device='cuda:0')
first norms: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], device='cuda:0')


In [5]:
imgs, targets, _ = next(iter(loader))
print("raw:", imgs.shape, imgs.min(), imgs.max())

proc = model.preprocess()
pimgs = [proc(ToPILImage()(im)) for im in imgs[:4]]
batch = torch.stack(pimgs)
print("pre:", batch.shape, batch.min(), batch.max())

raw: torch.Size([16, 3, 224, 224]) tensor(-2.1179) tensor(2.6400)
pre: torch.Size([4, 3, 224, 224]) tensor(-2.1179) tensor(2.6400)


In [6]:
logits = model.forward_batch(imgs[:4].to(device))
print("logits:", logits.shape,
      "min/max/mean:", logits.min(), logits.max(), logits.mean())

top5 = np.argsort(-logits, axis=1)[:4,:5]
for i in range(4):
    print(f"sample {i} top5:", top5[i])

logits: (4, 1000) min/max/mean: 41.89975 73.410225 57.78988
sample 0 top5: [632 714 715 545 452]
sample 1 top5: [632 641 781 577 920]
sample 2 top5: [919 632 818 714 971]
sample 3 top5: [919 489 616 490 824]


In [9]:
print(imagenet_classes)
print(len(imagenet_classes))

['tench', 'goldfish', 'great white shark', 'tiger shark', 'hammerhead shark', 'electric ray', 'stingray', 'rooster', 'hen', 'ostrich', 'brambling', 'goldfinch', 'house finch', 'junco', 'indigo bunting', 'American robin', 'bulbul', 'jay', 'magpie', 'chickadee', 'American dipper', 'kite (bird of prey)', 'bald eagle', 'vulture', 'great grey owl', 'fire salamander', 'smooth newt', 'newt', 'spotted salamander', 'axolotl', 'American bullfrog', 'tree frog', 'tailed frog', 'loggerhead sea turtle', 'leatherback sea turtle', 'mud turtle', 'terrapin', 'box turtle', 'banded gecko', 'green iguana', 'Carolina anole', 'desert grassland whiptail lizard', 'agama', 'frilled-necked lizard', 'alligator lizard', 'Gila monster', 'European green lizard', 'chameleon', 'Komodo dragon', 'Nile crocodile', 'American alligator', 'triceratops', 'worm snake', 'ring-necked snake', 'eastern hog-nosed snake', 'smooth green snake', 'kingsnake', 'garter snake', 'water snake', 'vine snake', 'night snake', 'boa constrictor

In [10]:
probs = np.zeros((1, logits.shape[1]), dtype=np.float32)
idx   = imagenet_classes.index("hen")
probs[0, idx] = 1.0
print("mapped hen->", ds.decision_mapping(probs)[0])

mapped hen-> ['bird' 'truck' 'knife' 'oven' 'elephant' 'dog' 'clock' 'keyboard' 'chair'
 'cat' 'bottle' 'car' 'boat' 'bicycle' 'bear' 'airplane']


In [11]:
print("=== ImageNet top-5 per sample ===")
for i in range(min(4, logits.shape[0])):
    top5_idx = np.argsort(-logits[i])[:5]
    top5_names = [imagenet_classes[j] for j in top5_idx]
    print(f"sample {i:2d}:", top5_names)

=== ImageNet top-5 per sample ===
sample  0: ['music speaker', 'plectrum', 'Pickelhaube', 'electric fan', 'poke bonnet']
sample  1: ['music speaker', 'maraca', 'scoreboard', 'gong', 'traffic light']
sample  2: ['traffic or street sign', 'music speaker', 'spotlight', 'plectrum', 'bubble']
sample  3: ['traffic or street sign', 'chain-link fence', 'knot', 'chain mail', 'scarf']


In [12]:
import torch

logits_t = torch.from_numpy(logits)        #[B×1000]
probs_t = torch.softmax(logits_t, dim=1)   
probs = probs_t.numpy()                    #[B×1000]
mapped = ds.decision_mapping(probs)
print("mapped:", type(mapped), "dtype:", mapped.dtype, "shape:", mapped.shape)
print("first row:", mapped[0])
print("second row:", mapped[1])

print("\n=== Mapped cue-conflict predictions ===")
top1_names = mapped[:, 0]    #length-B array

for i, name in enumerate(top1_names[:4]):
    print(f"sample {i:2d}: {name}")

mapped: <class 'numpy.ndarray'> dtype: <U8 shape: (4, 16)
first row: ['bottle' 'clock' 'keyboard' 'car' 'bird' 'dog' 'airplane' 'truck' 'bear'
 'oven' 'chair' 'knife' 'elephant' 'bicycle' 'boat' 'cat']
second row: ['clock' 'bird' 'truck' 'car' 'dog' 'chair' 'elephant' 'bottle' 'boat'
 'oven' 'bear' 'cat' 'airplane' 'keyboard' 'knife' 'bicycle']

=== Mapped cue-conflict predictions ===
sample  0: bottle
sample  1: clock
sample  2: clock
sample  3: car


In [26]:
hyco = model.model
hyco.eval()
with torch.no_grad():
    feats1 = hyco.encode_image(batch[:5].to(device), project=True)
    feats2 = hyco.encode_image(batch[:5].clone().to(device), project=True)
    print("identical? ", torch.allclose(feats1, feats2))
    # check variability
    d = (feats1[0] - feats1[1]).norm().item()
    print("feature distance between sample 0 & 1:", d)


identical?  True
feature distance between sample 0 & 1: 0.1923612803220749


In [13]:
import torch, gc
from modelvshuman.utils import load_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gc.collect()
torch.cuda.empty_cache()

model_wrapper, _ = load_model("clip")
ds = load_dataset("cue-conflict", batch_size=16, num_workers=16)
loader = ds.loader

model_wrapper.model.to(device)
model_wrapper.model.eval()



100%|██████████| 1000/1000 [00:43<00:00, 23.19it/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [14]:
!python -m modelvshuman \
    -m hycoclip \
    -d cue-conflict \
    -b 64 \
    -w 8


Evaluating model hycoclip on dataset cue-conflict using Pytorch Evaluator








accuracy (top-1): 6.25




c:\Users\xjzb2\.conda\envs\hycoclip\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

  0%|          | 0/20 [00:00<?, ?it/s]WARNING:tensorflow:From c:\users\xjzb2\compo_learning\model-vs-human\modelvshuman\models\wrappers\tensorflow.py:5: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

















100%|██████████| 20/20 [02:12<00:00,  6.63s/it]


In [15]:
!python -m modelvshuman \
    -m clip \
    -d cue-conflict \
    -b 64 \
    -w 8


Evaluating model clip on dataset cue-conflict using Pytorch Evaluator








accuracy (top-1): 44.06





100%|██████████| 1000/1000 [00:40<00:00, 24.99it/s]

  0%|          | 0/20 [00:00<?, ?it/s]WARNING:tensorflow:From c:\users\xjzb2\compo_learning\model-vs-human\modelvshuman\models\wrappers\tensorflow.py:5: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.

















100%|██████████| 20/20 [02:00<00:00,  6.01s/it]
